In [1]:
# just import statements
import time
import numpy as np
import simple_wta, convex_relaxation, regret_matching, spectral_clustering, ahuja

In [82]:
# creating a random WTA problem
n_weapons = 300
n_targets = 175
prob = simple_wta.random_wta_factory(n_weapons, n_targets)
print(type(prob))

# target values are stored in WTAProblem.v
print(np.shape(prob.v))

# pk data is stored in WTAProblem.p
print(np.shape(prob.p))

# specific WTA problems can be constructed by passing a
# value vector and pk matrix to the class constructor
copy_problem = simple_wta.WTAProblem(prob.v.copy(),prob.p.copy())

<class 'simple_wta.WTAProblem'>
(175,)
(300, 175)


In [83]:
# An upper bound on the optimal WTA objective can be found 
# by replacing the WTA problem with a knapsack problem, and optimizing
# in greedy fashion.

# this is implemented according to the Ahuja paper

# "combinatorial lower-bounding (clb) maximum marginal return (mmr) algorithm"
t0 = time.perf_counter()
x = simple_wta.clb_mmr_alg(prob)
t1 = time.perf_counter()
print("solution found in %.3f seconds." % (t1-t0))
print("objective value %.1f" % prob.objective(x))

solution found in 0.003 seconds.
objective value 318.5


In [84]:
# now gonna fuck with the ahuja code
t0 = time.perf_counter()
x = ahuja.optimize(prob)
t1 = time.perf_counter()
print("solution found in %.3f seconds." % (t1-t0))
print("objective value %.1f" % prob.objective(x))

solution found in 88.913 seconds.
objective value 22.0


In [73]:
# We can use a multi-agent reinforcement learning algorithm
# directly on the original problem.

# Run regret-matching for 50 rounds and sample a random action profile
# from a probability distribution derived from the regret vectors.
rounds = 50
t0 = time.perf_counter()
x,P,R = regret_matching.learning_dynamics(prob, rounds)
t1 = time.perf_counter()

print("solution found in %.1f seconds." % (t1-t0))
print("solution time divided by # weapons %.1f" % ((t1-t0)/n_weapons))
print("mixed strategy objective value %.1f" % prob.mixed_objective(P))
print("argmax stragey objective value %.1f" % prob.objective(np.argmax(P,1)))

# this is an improvement over the maximum-marginal-return approach
# but comes at significant computaitonal cost.

# However, in practice the bulk of the computations are done in parallel
# (each weapon only computes their own regret).

KeyboardInterrupt: 

In [74]:
# now we will demonstrate scaling improvements afforded by
# partitioning the large WTA problem into smaller subproblems.

# first we will do this randomly.

n_clusters = int(np.sqrt(n_weapons))
sub_problems, coalitions, missions = spectral_clustering.random_reduction(prob, n_clusters)

t0 = time.perf_counter()
rounds = 150
actions = [regret_matching.learning_dynamics(prob,rounds)[1] for prob in sub_problems]
value1 = np.sum([sub_problems[i].mixed_objective(actions[i]) for i in range(n_clusters)])
value2 = np.sum([sub_problems[i].objective(np.argmax(actions[i],1)) for i in range(n_clusters)])
t1 = time.perf_counter()

print("solution found in %.1f seconds." % (t1-t0))
print("solution time divided by (# weapons) %.3f" % ((t1-t0)/(n_weapons)))
print("mixed strategy objective value %.1f" % value1)
print("argmax stragey objective value %.1f" % value2)

# its apparent that the divide-and-conquer strategy drastically accelerates
# learning

solution found in 33.0 seconds.
solution time divided by (# weapons) 0.132
mixed strategy objective value 133.9
argmax stragey objective value 133.7


In [75]:
# finally we will demonstrate the objective improvements when spectral clustering is applied

t0 = time.perf_counter()
sub_problems, coalitions, missions = spectral_clustering.reduce_problem(prob, n_clusters)
rounds = 150
actions = [regret_matching.learning_dynamics(prob,rounds)[1] for prob in sub_problems]
value1 = np.sum([sub_problems[i].mixed_objective(actions[i]) for i in range(n_clusters)])
value2 = np.sum([sub_problems[i].objective(np.argmax(actions[i],1)) for i in range(n_clusters)])
t1 = time.perf_counter()

print("solution found in %.1f seconds." % (t1-t0))
print("solution time divided by (# weapons) %.3f" % ((t1-t0)/(n_weapons)))
print("mixed strategy objective value %.1f" % value1)
print("argmax stragey objective value %.1f" % value2)

# obviously, spectral clustering is better than random clustering

solution found in 37.6 seconds.
solution time divided by (# weapons) 0.150
mixed strategy objective value 70.0
argmax stragey objective value 68.1


In [87]:
# what about ahuja algorithm on random clusters?
t0 = time.perf_counter()
sub_problems, coalitions, missions = spectral_clustering.random_reduction(prob, n_clusters)
rounds = 100
actions = [ahuja.optimize(prob) for prob in sub_problems]
value = np.sum([sub_problems[i].objective(actions[i]) for i in range(n_clusters)])
t1 = time.perf_counter()

print("solution found in %.1f seconds." % (t1-t0))
print("value %.1f" % value)

solution found in 1.6 seconds.
value 47.9


In [88]:
# what about ahuja algorithm on spectral clusters? 

t0 = time.perf_counter()
sub_problems, coalitions, missions = spectral_clustering.reduce_problem(prob, n_clusters)
actions = [ahuja.optimize(prob) for prob in sub_problems]
value = np.sum([sub_problems[i].objective(actions[i]) for i in range(n_clusters)])
t1 = time.perf_counter()
print("solution found in %.3f seconds." % (t1-t0))
print("objective value %.1f" % value)

c:\Python39\lib\site-packages\numpy\lib\function_base.py:959: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


solution found in 3.136 seconds.
objective value 29.7


In [98]:
"""
I'm going to compare solution quality and run time of several different
methods at various problem scales
"""

# number of clusters 
L = np.array([10,12,14,16])
# number of weapons
N = L**2
# number of targets
M = np.array(0.75*N,dtype=int)
# WTA problems
problems = [simple_wta.random_wta_factory(N[i],M[i]) for i in range(len(N))]
# baseline performance for comparison
baseline = [problems[i].objective(simple_wta.clb_mmr_alg(problems[i])) for i in range(len(N))]
f1 = np.zeros(len(N))          # objective value for ahuja's method on full problem
f2 = np.zeros((len(N),100))    # objective value for ahuja's method on 100 random coalitions
f3 = np.zeros(len(N))          # objective value for ahuja's method with spectral clustering
# runnign times
t1 = np.zeros(len(N))
t2 = np.zeros((len(N),100))
t3 = np.zeros(len(N))

for i in range(len(problems)):
    print(i)
    tstart = time.perf_counter()
    f1[i] = problems[i].objective(ahuja.optimize(problems[i]))
    t1[i] = time.perf_counter()-tstart

    for j in range(100):
        print(j)
        tstart = time.perf_counter()
        sub_problems, coalitions, missions = spectral_clustering.random_reduction(problems[i], L[i])
        f2[i,j] = np.sum([p.objective(ahuja.optimize(p)) for p in sub_problems])
        t2[i,j] = time.perf_counter()-tstart
    
    tstart = time.perf_counter()
    sub_problems, coalitions, missions = spectral_clustering.reduce_problem(problems[i], L[i])
    f3[i] = np.sum([p.objective(ahuja.optimize(p)) for p in sub_problems])
    t3[i] = time.perf_counter()-tstart


0


c:\Python39\lib\site-packages\numpy\lib\function_base.py:959: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


NodeNotFound: Source 0 not in G